In [1]:
import math
import re
import pandas as pd

In [2]:
train_df = pd.read_csv('../ass5/train.csv')
val_df = pd.read_csv('../ass5/validation.csv')
test_df = pd.read_csv('../ass5/test.csv')

In [4]:
def extract_sentences(df, column="sentence"):
    """Extract plain text sentences from the specified column."""
    # Drop NaN and strip extra spaces
    return df[column].dropna().astype(str).str.strip().tolist()

# Extract lists of sentences
train_sentences = extract_sentences(train_df)
val_sentences   = extract_sentences(val_df)
test_sentences  = extract_sentences(test_df)

# Optional: print a few samples
print("Train samples:", train_sentences[:3])
print("Validation samples:", val_sentences[:3])
print("Test samples:", test_sentences[:3])


Train samples: ['યુનાઇટેડ સ્ટેટ્સ બ્યુરો ઓફ લેબર સ્ટેટિસ્ટિક્સ અનુસાર, 2008 માં અપરાધ દ્રશ્ય તપાસકર્તાઓ માટેનો સરેરાશ પગાર $ 55,040 હતો, અથવા કલાક દીઠ 23.97 ડોલર હતો', 'શ્રદ્ધાએ શરૂઆતમાં સારી રમત બતાવી, પરંતુ પછી તેની રમત એકદમ ધીમી થઈ ગઈ', 'અને વિચારોને વધુ સ્પષ્ટતા આપવા માટે, સોવલો અને અલજીઝ સાથેના ફટકામાં પ્રકાશની રુન છીણી પર કાપી છે']
Validation samples: ['જે ધ્યાને લેતા દેશભરમાં નવીનતમ ટેકનોલોજીનો ઉપયોગ કરીને એકીસાથે ઓછા સમયમાં વધુ પ્રમાણમાં આવાસો બનાવવામાં આવે તેવું કેન્દ્ર સરકાર દ્વારા સુચન કરવામાં આવેલ છે', '11. આ રમત "સાપની"', 'એક માદા સ્વપ્ન પુસ્તક એવી સ્ત્રીને પ્રિય પ્રણય રજૂ કરે છે જે વોર્મ્સ ખાવવાનું સ્વપ્ન ધરાવે છે;']
Test samples: ['રાત્રીના સમયે એકલ દોકલ વ્યકિત દેખાય તો તેના વાહનને આંતરી છરી બતાવી માલમત્તા લૂંટી લેવાની આદત છે', 'હાલમાં જ આવેલા એક રિપોર્ટમાં આ વાત કહેવામાં આવી છે', 'જલદી કણક જાડા અને એકરૂપ બની જાય છે, વેનીલા એસેન્સમાં રેડવું અને ફરીથી ફરીથી મિશ્રણ કરો']


In [5]:
def tokenize(sentence):
    """
    Splits a sentence into words, converts to lowercase, and removes punctuation.
    """
    # Ensure the input is a string
    if not isinstance(sentence, str):
        return []
    tokens = re.sub(r'[^\w\s]', '', sentence.lower()).split()
    return tokens

In [6]:
def compute_idf(train_corpus_tokens):
    """
    Calculates IDF scores for a vocabulary learned from the training corpus.
    :param train_corpus_tokens: A list of tokenized sentences from the training data.
    :return: A tuple containing (vocabulary, idf_scores_dict).
    """
    num_docs = len(train_corpus_tokens)
    
    # Build vocabulary and document frequency (DF) in one pass
    vocab = set()
    doc_freq = {}
    for doc in train_corpus_tokens:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            vocab.add(word)
            doc_freq[word] = doc_freq.get(word, 0) + 1
            
    sorted_vocab = sorted(list(vocab))
    
    # Calculate IDF scores using the document frequencies
    idf_scores = {
        word: math.log(num_docs / (doc_freq.get(word, 0) + 1))
        for word in sorted_vocab
    }
    
    print(f"Learned vocabulary with {len(sorted_vocab)} words and calculated IDF scores.")
    return sorted_vocab, idf_scores

In [7]:
def compute_tf(sentence_tokens):
    """
    Calculates TF scores for a single tokenized sentence.
    :param sentence_tokens: A list of words for one sentence.
    :return: A dictionary of {word: tf_score}.
    """
    doc_len = len(sentence_tokens)
    if doc_len == 0:
        return {}
        
    word_counts = {}
    for word in sentence_tokens:
        word_counts[word] = word_counts.get(word, 0) + 1
        
    tf_scores = {
        word: count / doc_len
        for word, count in word_counts.items()
    }
    return tf_scores

In [8]:
tokenized_train = [tokenize(s) for s in train_sentences]
tokenized_val = [tokenize(s) for s in val_sentences]
tokenized_test = [tokenize(s) for s in test_sentences]

In [9]:
vocabulary, idf_values = compute_idf(tokenized_train)
word_to_index = {word: i for i, word in enumerate(vocabulary)}

Learned vocabulary with 5477 words and calculated IDF scores.


In [10]:
# (Keep all your other functions like tokenize, compute_idf, compute_tf the same)

def create_tfidf_vectors(corpus_tokens, vocabulary, idf_scores, word_map):
    """
    Creates a memory-efficient sparse representation of TF-IDF vectors.
    Instead of a list of lists, it returns a list of dictionaries.
    """
    tfidf_vector_list = []
    
    for doc_tokens in corpus_tokens:
        tf_scores_doc = compute_tf(doc_tokens)
        
        # This dictionary will only store non-zero values for this sentence.
        sparse_vector = {} 
        
        for word, tf_val in tf_scores_doc.items():
            if word in word_map:
                index = word_map[word]
                # Store the TF-IDF score with its index as the key
                sparse_vector[index] = tf_val * idf_scores[word]
                
        tfidf_vector_list.append(sparse_vector)
        
    return tfidf_vector_list

In [11]:
print("\nVectorizing all datasets using a sparse representation...")
X_train_tfidf_sparse = create_tfidf_vectors(tokenized_train, vocabulary, idf_values, word_to_index)
X_val_tfidf_sparse = create_tfidf_vectors(tokenized_val, vocabulary, idf_values, word_to_index)
X_test_tfidf_sparse = create_tfidf_vectors(tokenized_test, vocabulary, idf_values, word_to_index)
print("...Done.")

print("\nSparse Vectorization Summary")
print(f"Number of training vectors: {len(X_train_tfidf_sparse)}")
print(f"Number of validation vectors: {len(X_val_tfidf_sparse)}")
print(f"Number of testing vectors: {len(X_test_tfidf_sparse)}")

print(f"\nFirst sparse vector (train): {X_train_tfidf_sparse[0]}")


Vectorizing all datasets using a sparse representation...
...Done.

Sparse Vectorization Summary
Number of training vectors: 1238
Number of validation vectors: 154
Number of testing vectors: 156

First sparse vector (train): {3881: 0.27948283794280854, 4813: 0.27948283794280854, 3354: 0.2493460040054196, 931: 0.19810013459926107, 4227: 0.27948283794280854, 4814: 0.27948283794280854, 326: 0.2015802523241974, 40: 0.2618539201989753, 3525: 0.191878358788797, 335: 0.2618539201989753, 2359: 0.2618539201989753, 2155: 0.27948283794280854, 3592: 0.21920917006803067, 5121: 0.2618539201989753, 2708: 0.2493460040054196, 98: 0.27948283794280854, 5294: 0.1391123074245474, 277: 0.16632241683653098, 1239: 0.19810013459926107, 2290: 0.27948283794280854, 57: 0.27948283794280854, 2084: 0.225014882790836}
